## FOR + NUMPY

In [ ]:
import numpy as np

# Matrices aleatorias grandes
N = 1000
A = np.random.rand(N, N)
B = np.random.rand(N, N)
C = np.zeros((N, N))

# Multiplicación de matrices usando un bucle for
for i in range(N):
    for j in range(N):
        for k in range(N):
            C[i, j] += A[i, k] * B[k, j]

# Matrices aleatorias grandes
N = 1000
A = np.random.rand(N, N)
B = np.random.rand(N, N)

# Multiplicación de matrices vectorizada
C = np.dot(A, B)

## JOPLIB

In [ ]:
import numpy as np
from joblib import Parallel, delayed

# Matrices aleatorias grandes
N = 1000
A = np.random.rand(N, N)
B = np.random.rand(N, N)

# Función que calcula una fila de la matriz resultado
def compute_row(i):
    row = np.zeros(N)
    for j in range(N):
        for k in range(N):
            row[j] += A[i, k] * B[k, j]
    return row

# Paralelización del cálculo de cada fila de la matriz C
results = Parallel(n_jobs=-1)(delayed(compute_row)(i) for i in range(N))

# Combinar los resultados en la matriz C final
C = np.vstack(results)


## MULTIPROCESSING

In [ ]:
import numpy as np
import multiprocessing as mp

# Matrices aleatorias grandes
N = 1000
A = np.random.rand(N, N)
B = np.random.rand(N, N)

# Función que calcula una fila de la matriz resultado
def compute_row(i):
    row = np.zeros(N)
    for j in range(N):
        for k in range(N):
            row[j] += A[i, k] * B[k, j]
    return row

# Función para recoger los resultados
def collect_result(result, queue):
    queue.put(result)

# Crear una cola para recoger los resultados de los procesos
queue = mp.Queue()

# Crear y lanzar procesos
processes = []
for i in range(N):
    p = mp.Process(target=collect_result, args=(compute_row(i), queue))
    processes.append(p)
    p.start()

# Recoger los resultados
results = []
for _ in range(N):
    results.append(queue.get())

# Esperar a que todos los procesos terminen
for p in processes:
    p.join()

# Combinar los resultados en la matriz C final
C = np.vstack(results)


## Union de todo 

In [ ]:
import numpy as np
import time
import multiprocessing as mp

# Tamaño de la matriz
N = 100

# Matrices aleatorias grandes
A = np.random.rand(N, N)
B = np.random.rand(N, N)

# --- FOR ORIGINAL ---
def for_loop():
    C = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                C[i, j] += A[i, k] * B[k, j]
    return C

# --- VECTORIZACIÓN ---
def vectorized():
    return np.dot(A, B)

# --- FUNCIONES PARA MULTIPROCESSING ---
def compute_row(i):
    row = np.zeros(N)
    for j in range(N):
        for k in range(N):
            row[j] += A[i, k] * B[k, j]
    return row

def parallel_multiprocessing():
    with mp.Pool(mp.cpu_count()) as pool:
        results = pool.map(compute_row, range(N))
    return np.vstack(results)

# --- TIMEAR IMPLEMENTACIONES ---
def time_it(func, label):
    start = time.time()
    func()
    end = time.time()
    print(f"{label} tomó {end - start:.4f} segundos")

if __name__ == "__main__":
    # Timear las distintas implementaciones
    time_it(for_loop, "For Loop")
    time_it(vectorized, "Vectorización")
    time_it(parallel_multiprocessing, "Multiprocessing")


## last task

In [ ]:
import numpy as np
import time
from joblib import Parallel, delayed
import multiprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Función para calcular tiempo de ejecución de un for loop tradicional
def time_for_loop(N, A, B):
    start = time.time()
    C = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                C[i, j] += A[i, k] * B[k, j]
    end = time.time()
    return end - start

# Función para calcular tiempo de ejecución con vectorización usando numpy
def time_vectorization(N, A, B):
    start = time.time()
    C = np.dot(A, B)
    end = time.time()
    return end - start

# Función para calcular tiempo de ejecución con joblib (paralelización)
def time_joblib(N, A, B):
    def compute_row(i):
        C[i, :] = np.dot(A[i, :], B)
    C = np.zeros((N, N))
    start = time.time()
    Parallel(n_jobs=-1)(delayed(compute_row)(i) for i in range(N))
    end = time.time()
    return end - start

# Función para calcular tiempo de ejecución con multiprocessing
def time_multiprocessing(N, A, B):
    def compute_row(i):
        return np.dot(A[i, :], B)

    start = time.time()
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        C = pool.map(compute_row, range(N))
    end = time.time()
    return end - start

# Función para realizar simulaciones
def run_simulations(N, num_simulations=10):
    A = np.random.rand(N, N)
    B = np.random.rand(N, N)

    times_for = []
    times_vec = []
    times_joblib = []
    times_mp = []

    for _ in range(num_simulations):
        times_for.append(time_for_loop(N, A, B))
        times_vec.append(time_vectorization(N, A, B))
        times_joblib.append(time_joblib(N, A, B))
        times_mp.append(time_multiprocessing(N, A, B))

    return times_for, times_vec, times_joblib, times_mp

# Función para calcular estadísticas básicas
def calculate_statistics(times):
    df = pd.DataFrame({
        'for_loop': times[0],
        'vectorization': times[1],
        'joblib': times[2],
        'multiprocessing': times[3]
    })

    stats = df.describe()
    return df, stats

# Función para graficar los resultados
def plot_results(df):
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=df)
    plt.title('Comparación de tiempos de ejecución')
    plt.ylabel('Tiempo (segundos)')
    plt.show()

# Función principal que corre todo el proceso
def main(N=500, num_simulations=5):
    # Ejecutar simulaciones
    times_for, times_vec, times_joblib, times_mp = run_simulations(N, num_simulations)

    # Calcular estadísticas
    df, stats = calculate_statistics([times_for, times_vec, times_joblib, times_mp])
    print("Estadísticas de los tiempos de ejecución:")
    print(stats)

    # Graficar resultados
    plot_results(df)

# Ejecutar la función principal
main(N=500, num_simulations=10)
